<a href="https://colab.research.google.com/github/tourihasi/Openstudio/blob/main/%E8%A8%AD%E5%82%99%E6%83%85%E5%A0%B1%E5%BC%95%E7%B6%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 0) インストール & インポート（既に入っていれば pip はスキップされます）
try:
    import openstudio as osd
except ModuleNotFoundError:
    # Colab では先頭に '!' が必要
    !pip -q install openstudio
    import openstudio as osd

from google.colab import files

# ---- ユーティリティ ----
def load_model(path: str) -> osd.model.Model:
    """OSM を読み込んで Model を返す。失敗時は RuntimeError。"""
    opt_m = osd.model.Model.load(osd.path(path))
    if opt_m.is_initialized():
        return opt_m.get()
    raise RuntimeError(f"OpenStudio Model の読み込みに失敗: {path}")

def pick_uploaded_osm(upload_result: dict, label: str) -> str:
    """アップロード結果から最初の .osm を選ぶ。"""
    assert upload_result, f"{label}: ファイルがアップロードされていません。"
    cands = [n for n in upload_result.keys() if n.lower().endswith(".osm")]
    assert cands, f"{label}: 拡張子 .osm のファイルが見つかりません。"
    return cands[0]  # ラベルに依らず、最初の .osm を採用

# 1) 供与側（HVACあり）をアップロード
print("引き継ぐモデルデータを選択してください（HVACあり）")
uploaded_src = files.upload()
src = pick_uploaded_osm(uploaded_src, "Source")
print("Source OSM :", src)

# 2) 受け側（gbXML取込後）をアップロード
print("システムを付加するモデルを選択してください（gbXML取込後）")
uploaded_dst = files.upload()
dst = pick_uploaded_osm(uploaded_dst, "Target")
print("Target OSM :", dst)

# 3) モデル読み込み
src_m = load_model(src)
dst_m = load_model(dst)

# 4) ループ（Plant/Air）の複製のみ
for pl in src_m.getPlantLoops():
    _ = pl.clone(dst_m)
    print("[Clone] PlantLoop:", pl.nameString())

for al in src_m.getAirLoopHVACs():
    _ = al.clone(dst_m)
    print("[Clone] AirLoopHVAC:", al.nameString())

# 5) VRF 親機を clone & マップ化（重複 clone を避ける）
vrf_map = {}  # key: src_handle(str) -> value: dst_vrf(ModelObject)
for vrf_src in src_m.getAirConditionerVariableRefrigerantFlows():
    opt_vrf_dst_obj = vrf_src.clone(dst_m).to_AirConditionerVariableRefrigerantFlow()
    if not opt_vrf_dst_obj.is_initialized():
        print("[Warn] VRF の型変換に失敗:", vrf_src.nameString())
        continue
    vrf_dst = opt_vrf_dst_obj.get()
    vrf_map[str(vrf_src.handle())] = vrf_dst
    print("[Clone] VRF (outdoor):", vrf_src.nameString(), "->", vrf_dst.nameString())

# 6) VRF 端末（室内機）を clone & 親に再接続
for term_src in src_m.getZoneHVACTerminalUnitVariableRefrigerantFlows():
    opt_term_dst_obj = term_src.clone(dst_m).to_ZoneHVACTerminalUnitVariableRefrigerantFlow()
    if not opt_term_dst_obj.is_initialized():
        print("[Warn] VRF 端末の型変換に失敗:", term_src.nameString())
        continue
    term_dst = opt_term_dst_obj.get()

    # 親 VRF（供与側）を取得し、ハンドルで対応する複製先に接続
    parent_opt = term_src.airConditionerVariableRefrigerantFlow()  # Optional
    if parent_opt.is_initialized():
        parent_src = parent_opt.get()
        parent_dst = vrf_map.get(str(parent_src.handle()))
        if parent_dst:
            ok = parent_dst.addTerminal(term_dst)
            if ok:
                print("[Link ]", term_dst.nameString(), "->", parent_dst.nameString())
            else:
                print("[Warn] 端末の親接続に失敗:", term_dst.nameString())
        else:
            print("[Warn] 対応する親VRFが見つかりません:", term_src.nameString())
    else:
        print("[Warn] 端末に親VRFが設定されていません:", term_src.nameString())

# 7) その他ゾーン機器（複製のみ）
for fc in src_m.getZoneHVACFourPipeFanCoils():
    _ = fc.clone(dst_m)
    print("[Clone] FanCoil:", fc.nameString())

for ptac in src_m.getZoneHVACPackagedTerminalAirConditioners():
    _ = ptac.clone(dst_m)
    print("[Clone] PTAC:", ptac.nameString())

for pthp in src_m.getZoneHVACPackagedTerminalHeatPumps():
    _ = pthp.clone(dst_m)
    print("[Clone] PTHP:", pthp.nameString())

# 8) 保存 & ダウンロード
OUT = "target_with_HVAC.osm"
dst_m.save(osd.path(OUT), True)
print("Saved:", OUT)

files.download(OUT)
# ======================================================================
